# Content

>* [**Data preperation**](#0)
>* [**1. Data Quality**](#1)
    - [**Data Integrity**](#1_1)
    - [**Data fitness**](#1_2)
>* [**2. Plausibilitätschecks**](#2)
>* [**3. Analysis**](#2)
>* [**data preperation**](#3)


In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

---
## Data preperation <a name=0></a>

In [ ]:
#Den Header als Index zu verwenden
df = pd.read_csv("./Revision2/employees_satisfaction.csv", index_col = 0)

In [ ]:
#der Index ist rein numerisch.
df = pd.read_csv("./Revision2/employees_satisfaction.csv")

In [ ]:

display(df.head(5))

In [ ]:
df.info()

In [ ]:
df['age'].dtype

In [ ]:
df.columns.values

In [ ]:
df.describe(include='all')


---
## Data Quality  <a name=1></a>

### Data Integrity  <a name=1_1></a>

Data integrity is based on the following **metrics**: 

- [***Necessary, but not sufficient***](#a)
    - [Of known provence.](#a)
    - [Well-Annotated.](#a)
- ***Important***
    - [Timely.](#b)
    - [Complete.](#c)
    - [High Volume.](#d)
    - [Multivariant.](#e)
    - [Atomic](#f)
- ***Achievable***
    - [Consistent.](#g)
    - [Clear.](#h)
    - [Dimensionally structed.](#i)

### **Timely** <a name=b></a>

##### analysis

In [4]:
#Timely
print(df['entry_date'].min(), " - ", df['entry_date'].max())

NameError: name 'df' is not defined

##### fixes

### **complete** <a name=c></a>

##### analysis

In [ ]:
#Anzahl fehlende Werte pro spalte
df.isnull().sum()

In [ ]:
#Zeilen mit fehlenden werten anzeigen
df[df['Ernaehrung'].isnull()]

##### fixes

In [ ]:
#Schätzung der fehlenden Werte??

#Ist die durchschnittlichen Entfernungen der Mitglieder aus dem gleichen Stadtteil?
df.groupby('stadtteil')['entfernung'].apply(list)
#low variance means that the data points tend to be very close to the mean, and thus, the mean is a good measure of central tendency.
df.groupby('stadtteil')['entfernung'].var()

In [ ]:
#Fehlende WErten ersetzen:

# Angenommen, 'df' ist Ihr DataFrame und 'Stadtteil', 'Entfernung' sind die Spaltennamen
# Ersetzen Sie diese durch die tatsächlichen Namen in Ihrem Datensatz

# Berechnen Sie den Durchschnitt der Entfernungen für jeden Stadtteil
average_distances = df.groupby('Stadtteil')['Entfernung'].mean()

# Definieren Sie eine Funktion, die die fehlenden Entfernungen ersetzt
def impute_distance(row):
    if pd.isnull(row['Entfernung']):
        return average_distances[row['Stadtteil']]
    else:
        return row['Entfernung']

# Wenden Sie die Funktion auf jede Zeile an
df['Entfernung'] = df.apply(impute_distance, axis=1)


In [ ]:
df.fillna(df.mean(), inplace=True)  # Für den Durchschnitt
df.fillna(df.median(), inplace=True)  # Für den Median
df.fillna(df.mode().iloc[0], inplace=True)  # Für den Modus


### **High Volume.** <a name=d></a>

##### analysis

In [ ]:
#is the number of data in your dataset is enough?
len(df)

In [ ]:
#Is the number of data for each category in your dataset is enough?
df['Stadtteil'].value_counts()
df.groupby('stadtteil')['entfernung'].mean()

In [7]:
participation_rate = (111 / 256) * 100
print(f"Die Teilnahmequote beträgt {participation_rate:.3f}%")

##### fixes

### **multivariate** <a name=e></a>

##### analysis

In [ ]:
#multivariate
# Check the number of variables (columns) in the dataset
if df.shape[1] > 1:
    print("Der Datensatz ist multivariant mit ", df.shape[1], " Variablen.")
else:
    print("Der Datensatz ist nicht multivariant.")

In [ ]:
# Nur numerische Spalten auswählen
numerical_df = df.select_dtypes(include=[np.number])

# Korrelationsmatrix erstellen
corr = numerical_df.corr()

# Korrelationsmatrix anzeigen
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

##### fixes

### **atomic** <a name=f></a>

##### analysis

In [ ]:
#atomic
def is_atomic(df):
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, (list, dict))).any():
            return False
    return True

#überprüft,ob ein Element in der Spalte eine Instanz einer Liste oder eines dictionarys ist
print(is_atomic(df))

'''
# Assuming 'info' column contains dictionaries
info_df = df['info'].apply(pd.Series)
df = pd.concat([df.drop(['info'], axis=1), info_df], axis=1)


# Assuming 'tags' column contains lists
for tag in set(x for l in df['tags'] for x in l):
    df['tag_' + tag] = df['tags'].apply(lambda x: tag in x)

'''

In [8]:
unique_values = df['MA'].unique()
unique_values


    
df[df['id'].duplicated()]['id']

##### fixes

### **consistent** <a name=g></a>

##### analysis

In [ ]:
# consistent

#are there duplicates?

if(df.duplicated().any()):
    print("this dataset contains ", df.duplicated().sum() ," duplicates.")
    print(df[df.duplicated()])
else: print("this dataset does not contain duplicates.")

#you can drop duplicates as follows:
#df_dss.drop_duplicates() 

#are the decriptives used for the same value in the dataset consistent? e.g. spelling of "male" vs "Male"
#find inconsistencies:
df['gender'].value_counts()

In [ ]:
# Überprüfe die Einzigartigkeit der Werte in jeder Spalte
for column in df.columns:
    print(f"{column}: {df[column].nunique()} unique values")

##### fixes

In [ ]:
#you can drop duplicates as follows:
df.drop_duplicates() 

In [ ]:
#recoding

gender_recoding = {
"f": "Female",
"m": "Male"
}
df["gender"] = df["gender"].replace(gender_recoding)

### **clear** <a name=h></a>

##### analysis

In [ ]:
#clear

# Check column names for clarity
print("Column names:")
print(df.columns)

# Check unique values in each column
for column in df.columns:
    print(f"\nUnique values in {column}:")
    print(df[column].unique())

##### fixes

In [ ]:
# spalten umbenennen
df.rename(columns = {"Dept":"department"}, inplace = True)

In [ ]:
#recoding

gender_recoding = {
"f": "Female",
"m": "Male"
}
df["gender"] = df["gender"].replace(gender_recoding)

### **Dimensionally structed.** <a name=i></a>

##### analysis

In [ ]:
#Dimensionally structed.

# Überprüfe die ersten paar Zeilen des Datensatzes, um die Struktur zu sehen
df.head()

# Überprüfe die Anzahl der Dimensionen (Spalten) in Ihrem Datensatz
print("Number of dimensions in the dataset: ", df.shape[1])

In [ ]:
# Überprüfen Sie die Datentypen Ihrer Spalten
print(df.dtypes)

##### fixes

### Data Fit  <a name=1_2></a>

In [ ]:
#Representativeness
# Überprüfen Sie die Verteilung der Zielvariable 'satisfied'
print(df['satisfied'].value_counts(normalize=True))

## 1. data preperation <a name=1></a>

In [ ]:
#Den Header als Index zu verwenden
df = pd.read_csv("./Revision2/employees_satisfaction.csv", index_col = 0)

In [ ]:
#der Index ist rein numerisch.
df = pd.read_csv("./Revision2/employees_satisfaction.csv")

In [ ]:

display(df.head(5))

In [ ]:
df.info()

In [ ]:
df['age'].dtype

In [ ]:
df.columns.values

In [ ]:
df.describe(include='all')


---
## 2. Plausibilitätschecks <a name=2></a>

In [9]:
#Bereichsprüfung: liegen die Werte in den Spalten innerhalb eines erwarteten Bereichs

df['alter'].min()>=9 & df['alter'].max()<=19

In [ ]:
#Vollständigkeitsprüfung: sind alle erforderlichen Daten vorhanden?

missing_values = df.isnull().sum()
print("Fehlende Werte in jeder Spalte:\n", missing_values)

In [ ]:
#Duplikatsprüfung: ob es sich um tatsächliche Duplikate handelt oder ob es einen Fehler bei der Datenerfassung gab?

duplicates = df.duplicated(subset=['id'], keep=False)
print("Anzahl der Duplikate in der Spalte 'ID':", duplicates.sum())
df[duplicates]

In [ ]:
#Konsistenzprüfung: Überprüfen Sie, ob die Daten in verschiedenen Spalten konsistent sind

id_dept_mapping = {
"HR":"HR",
"MKT":"Marketing",
"PUR":"Purchasing",
"SAL":"Sales",
"TECH":"Technology"
}

import re

# Ihre Lösung
regular_expression = "[A-Z]+" # <- fügen Sie hier den entsprechenden regulären Ausdruck ein

def check_id(emp_id, department):
    dept_id = "".join(re.findall(regular_expression, emp_id))
    return id_dept_mapping.get(dept_id) == department

# Ist der Check für alle Zeilen des Datensatzes erfolgreich?
df.apply(lambda x: check_id(x["emp_id"], x["department"]), axis=1).all()

In [ ]:
#
inconsistent_data = (df['Mo'] == 1) & (df['Favorit'].isnull())
print("Anzahl der inkonsistenten Daten:", inconsistent_data.sum())
df[inconsistent_data]

In [10]:
#Gültigkeitsprüfung: Überprüfen Sie, ob die Werte in den Spalten gültig sind.

invalid_values_stadtteil = ~df['Stadtteil'].isin(['Nord', 'Ost', 'Süd', 'West'])
print("Anzahl der ungültigen Werte in der Spalte 'Stadtteil':", invalid_values_stadtteil.sum())

invalid_values_favorit = ~df['favorit'].isin(['Kochen', 'Malen', 'Musik', 'Sport', 'Werken'])
print("Anzahl der ungültigen Werte in der Spalte 'favorit':", invalid_values_stadtteil.sum())

---
## 3. Analysis <a name=3></a>

### 1.1. Anzahl Spalten

In [ ]:
len(df.columns)

In [ ]:
len(df.keys())

In [ ]:
len(df.axes[1])

In [ ]:
len(df)

In [ ]:
len(df.index)

In [ ]:
len(df.axes[0])

## 3. Spalten transformieren  <a name=3></a>

In [ ]:
df = df.rename(columns={'Dept': 'department'})


In [ ]:
df['salary'] *= 0.9

In [ ]:
df['entry_date'] = pd.to_datetime(df['entry_date'], format='%Y-%m-%d')


In [ ]:

edu_recoding = {
    "UG": "Undergrad",
    "PG": "Postgrad"
}
df['education'] = df['education'].map(edu_recoding).fillna(df['education'])

In [ ]:
df[['age','entry_date']]

diff = created.year - df['entry_date'].dt.year

len(df[df['age']- diff < 16])

In [ ]:
df["gender"] = df["gender"].fillna("Unknown")


In [ ]:
df = df.drop("last_raise", axis=1)

In [ ]:
idx = df[df['certifications']==9].index
df.loc[idx, "certifications"] = 0

In [ ]:
df['emp_id'].unique()
df['emp_id'].nunique()
df[df['emp_id'].duplicated(keep=False)]
df.drop_duplicates(keep==False, subset=['emp_id'])

In [ ]:
# Ihre Lösung
males = df[df['gender']=='Male']
females = df[df['gender']=='Female']
males[males['age'] == males['age'].max()]['department']

In [ ]:
#In welcher Abteilung sind die meisten zufriedenen Angestellten?
satisfied = df[df['satisfied'] == 1]
satisfied['department'].value_counts().index[0]

In [ ]:
#Und in welcher Abteilung sind die Angestellten im Schnitt am zufriedensten?
df.groupby('department')['satisfied'].mean().sort_values(ascending=False).index[0]

## 4. Diagramme  <a name=4></a>

In [ ]:
# outliers:
df.hist(column='age', bins=20, color='blue')

for column in df.columns:
    # Check if the column is numeric for valid histogram plotting
    if pd.api.types.is_numeric_dtype(df[column]):
        df.hist(column)


In [ ]:
df['department'].value_counts().plot(kind='bar')
plt.title('Anzahl angestellte pro department')
plt.ylabel('Anzahl angestellte')  # Füge
plt.show()

In [ ]:
# Ihre Lösung

df.groupby(['department', 'gender'])\
.size()\
.unstack('department')\
.plot(kind='pie',subplots=True, figsize=(15,10), layout=(3, 3), autopct='%1.1f%%')

## 5. grouping  <a name=5></a>

1. `df.groupby('job_level')['age']`: This groups the DataFrame `df` by the column 'job_level' and then selects the 'age' column from the grouped data. The result is a Series with the 'job_level' as the index. You can then apply aggregate functions (like mean, sum, etc.) to this Series to get statistics for 'age' within each 'job_level'.

2. `df.groupby(['job_level', 'age'])`: This groups the DataFrame `df` by two columns: 'job_level' and 'age'. The result is a DataFrame with a MultiIndex ('job_level' and 'age'). You can then apply aggregate functions to this DataFrame to get statistics for each combination of 'job_level' and 'age'.

In summary, the first expression is used when you want to analyze the 'age' column within each 'job_level', while the second expression is used when you want to analyze the data for each combination of 'job_level' and 'age'.

In [ ]:
df.groupby([df['entry_date'].dt.year, 'recruitment_type']).size().unstack('recruitment_type').plot(kind='line')